In [1]:
%autosave 30

Autosaving every 30 seconds


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from sklearn.feature_selection import chi2

In [3]:
# Load Titanic Datset

df = sns.load_dataset('titanic')
df

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True
888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


### # Considering Categorical Features -- `sex`, `embarked`, `pclass` and the target variable `survived`.

In [5]:
df_cats = df[['sex', 'embarked', 'pclass', 'alone', 'survived']]
df_cats.head()

,sex,embarked,pclass,alone,survived
0,male,S,3,False,0
1,female,C,1,False,1
2,female,S,3,True,1
3,female,S,1,False,1
4,male,S,3,True,0


### # Performing Label Encoding:

In [6]:
encoders = {}
for col in df_cats.columns:
    encoders[col] = LabelEncoder()
    df_cats[col] = encoders[col].fit_transform(df_cats[col])

C:\Users\Admin\AppData\Local\Temp\ipykernel_15944\118350151.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cats[col] = encoders[col].fit_transform(df_cats[col])


In [7]:
encoders

{'sex': LabelEncoder(),
 'embarked': LabelEncoder(),
 'pclass': LabelEncoder(),
 'alone': LabelEncoder(),
 'survived': LabelEncoder()}

In [8]:
df_cats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   sex       891 non-null    int32
 1   embarked  891 non-null    int32
 2   pclass    891 non-null    int64
 3   alone     891 non-null    int64
 4   survived  891 non-null    int64
dtypes: int32(2), int64(3)
memory usage: 28.0 KB


### # Encoded Categorical Dataset:

In [9]:
df_cats.head()

,sex,embarked,pclass,alone,survived
0,1,2,2,0,0
1,0,0,0,0,1
2,0,2,2,1,1
3,0,2,0,0,1
4,1,2,2,1,0


In [10]:
## Train-Test Split

X = df_cats.drop(columns=['survived'])
y = df_cats[['survived']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [11]:
# Check Null vals in the training set

X_train.isna().sum()

sex         0
embarked    0
pclass      0
alone       0
dtype: int64

### # Performing Chi-Square test:

- **Goodness of Fit**<br>
To check how well theoretical distributions explain the observed ones.


- **Test of Independence**<br>
To test whether two categorical variables are related to one another.

**`Chi Square Test`**

Compute chi-squared stats between each non-negative feature and class.

This score can be used to select the n_features features with the
highest values for the test chi-squared statistic from X, which must
contain only non-negative features such as booleans or frequencies
(e.g., term counts in document classification), relative to the classes.

Recall that the chi-square test measures dependence between stochastic
variables, so using this function "weeds out" the features that are the
most likely to be independent of class and therefore irrelevant for
classification.

### Higher the Chi-Square stat, more important is the feature! Why tf?

**Ho:** The given two Variables are **independent**. Any dependency would be purely random.

**Ha:** There's some dependency between the two.

=> Now, back to the statement, higher **chi-square stat** would suggest to reject the null hypothesis and to discard the notion that the two variables are independent.

In [12]:
chi2_stats, pvals = chi2(X_train, y_train)
chi2_stats

array([60.41964418,  7.92931297, 30.38083393,  8.35778435])

In [13]:
df_cats.columns

Index(['sex', 'embarked', 'pclass', 'alone', 'survived'], dtype='object')

In [14]:
pvals

array([7.66442567e-15, 4.86402622e-03, 3.55018608e-08, 3.84038546e-03])

**=>** Null Hypo for each variabe is discarded (look at those cute p-values).

In [15]:
df_chi2 = pd.DataFrame(np.c_[df_cats.columns[:-1], chi2_stats, pvals], 
                       columns=["Categorical Variables", "Chi2 Stat", "p-value"])
df_chi2.set_index("Categorical Variables", inplace=True)

In [16]:
df_chi2.sort_values(by="Chi2 Stat", ascending=False)

,Chi2 Stat,p-value
Categorical Variables,,
sex,60.419644,0.0
pclass,30.380834,0.0
alone,8.357784,0.00384
embarked,7.929313,0.004864


**=>** This way we can choose as many categorical variables out of the availables with the highest `chi-square stat`.

And apparently, `Sex` is the most important feature as the deciding factor for the surviving status of people and that does make sense.